## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is, essentially, the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

* Describe RDDs as fundamental storage units in the Spark computing environment
* Create RDDs from Python collections
* Set number of partitions for parallelizing RDDs
* Review an RDD's dependency graph at different stages of processing. 
* Apply the map(func) transformation to a given function on all elements of an RDD in different partitions
* Use collect() action to trigger the processing stage of spark's lazy evaluation
* Use count() action to calculate the number of elements of a parallelized RDD
* Use filter(func) to filter unwanted data from RDDs
* Develop an understanding of Python's lambda functions for RDDs processing


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies will performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified.
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the Database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new data set from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base data set (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the Driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some of key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [2]:
data = list(range(1, 1001))
len(data)

#1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use 'local[*]' as the master.

In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `parallelize` method to create an rdd will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [3]:
rdd = sc.parallelize(data, 10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the getNumPartitions method.

In [4]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first n items in the RDD
* `top`: returns the top n items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [6]:
# count
rdd.count()

1000

In [7]:
# first
rdd.first()

1

In [8]:
# take
rdd.take(3)

[1, 2, 3]

In [9]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [10]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 be a random number from 0-1.

In [11]:
import random
import numpy as np

nums = np.array(range(1,1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([6.76978508e-01, 1.14652478e+00, 2.34005828e+00, 1.76012763e+00,
       3.37205906e+00, 9.23776160e-01, 1.08679486e+00, 7.81211018e+00,
       2.16025016e+00, 2.96061572e+00, 3.66392942e+00, 9.28606839e+00,
       1.13758675e+01, 7.46049715e-01, 1.18959031e+01, 3.08856219e-01,
       1.58608429e+01, 7.88945033e+00, 7.47415266e+00, 6.90741268e+00,
       8.90460333e+00, 1.81512996e+01, 1.00159747e+01, 1.22226154e+01,
       2.45596191e+00, 1.55200206e+01, 1.03587012e+01, 1.27838296e+01,
       3.61661006e+00, 2.54503778e+01, 5.55579279e+00, 2.16707374e+00,
       1.94090012e+01, 2.11130669e+01, 5.16082711e+00, 2.26845713e+01,
       3.15780161e+01, 2.70211532e+01, 7.38740431e+00, 1.40131525e+01,
       1.03395947e+01, 3.07288187e+01, 2.10196918e+00, 1.83636805e+01,
       1.40492763e+01, 1.35288422e+00, 4.30635976e+01, 1.07101302e+01,
       4.09472591e+01, 4.86377847e+01, 1.11854635e+01, 2.01756359e+00,
       4.19398340e+01, 1.82328331e+01, 4.38020901e+01, 3.25727537e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [19]:
price_items = sc.parallelize(sales_figures, 10)
price_items.take(10)

[0.6769785075949011,
 1.1465247770926903,
 2.3400582825067424,
 1.7601276300957172,
 3.372059061297186,
 0.9237761603494465,
 1.0867948625289854,
 7.812110182743091,
 2.160250157735538,
 2.9606157186921833]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax` that returns the amount of money our company will receive after the sale tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the map method and assign it to a variable `renenue_minus_tax`

In [13]:
def sales_tax(num):
    return num*0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [16]:
# perform action to retrieve rdd values
revenue_minus_tax.take(5)

[0.622820226987309,
 1.0548027949252752,
 2.152853619906203,
 1.6193174196880598,
 3.1022943363934115]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a map method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [29]:
discounted = revenue_minus_tax.map(lambda x: 0.9*x)

In [30]:
discounted.take(10)

[0.5605382042885781,
 0.9493225154327477,
 1.9375682579155828,
 1.4573856777192538,
 2.7920649027540705,
 0.7648866607693418,
 0.899866146174,
 6.46842723131128,
 1.7886871306050254,
 2.451389815077128]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff use a collection method to see the 15 costliest items.

In [20]:
price_items.map(sales_tax).map(lambda x: 0.9*x).top(15)

[795.8947346705804,
 794.0716773011138,
 777.8227540690676,
 768.9189361784306,
 759.9114694395467,
 750.88337717508,
 746.6799698616255,
 739.3356483259944,
 735.4107585507921,
 734.1693999682519,
 730.1253885708276,
 728.8776918198189,
 720.6872525817296,
 709.407215622977,
 699.399303966529]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [23]:
discounted.toDebugString()

b'(10) PythonRDD[13] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use flatMap rather than a simple map. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will a tuple with (original price, post-discount price).

In [28]:
mapped = price_items.map(lambda x: (x, x*0.92*0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.6769785075949011, 0.5605382042885781), (1.1465247770926903, 0.9493225154327477), (2.3400582825067424, 1.9375682579155828), (1.7601276300957172, 1.4573856777192538), (3.372059061297186, 2.7920649027540705), (0.9237761603494465, 0.7648866607693418), (1.0867948625289854, 0.899866146174), (7.812110182743091, 6.46842723131128), (2.160250157735538, 1.7886871306050254), (2.9606157186921833, 2.451389815077128)]


Note that we have 1000 tuples created to our specification. Let's take a look at how flatMap differs in its implementation. Use the `flatMap` method with the same function you created above.

In [33]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92*0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.6769785075949011, 0.5605382042885781, 1.1465247770926903, 0.9493225154327477, 2.3400582825067424, 1.9375682579155828, 1.7601276300957172, 1.4573856777192538, 3.372059061297186, 2.7920649027540705]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use flatmap rather than map in order to properly reduce to our specifications. This is not one of those instances, but int he upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the filter method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a filter function to meet the consultant's suggestion's specifications. set RDD = `selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [36]:
# use the filter function
selected_items = discounted.filter(lambda x: x > 300)

# calculate total remaining in inventory 
selected_items.count()

275

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use a reduce method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [35]:
selected_items.reduce(lambda x, y: x + y)

129276.04274246666

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [37]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1,50),x))

sales_data.take(7)

[(43, 316.2154250918928),
 (14, 306.2098793577082),
 (25, 331.1926574626218),
 (15, 320.1695691330134),
 (19, 340.816777253021),
 (21, 337.87040396305696),
 (7, 301.01230241822077)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called __reduceByKey__ to perform reducing operations while grouping by keys. After you have calculated the total, use the __sortBy__ method on the RDD to rank the users from the highest spending to the least spending.



In [40]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x + y)
total_spent.take(5)

[(50, 2797.5423042781067),
 (30, 2545.3976594708033),
 (10, 3459.531636861962),
 (20, 2254.878686883485),
 (40, 2194.97087897858)]

In [41]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1], ascending=False).collect()

[(32, 5275.74797898986),
 (26, 4952.329293283967),
 (1, 4928.566149988927),
 (7, 4925.347487739643),
 (14, 4256.99785903659),
 (34, 3966.1712428679884),
 (9, 3884.178797607057),
 (16, 3639.0774938594554),
 (42, 3466.762435926462),
 (10, 3459.531636861962),
 (25, 3447.3551446979754),
 (43, 3369.4440746874197),
 (46, 3284.2354016312775),
 (39, 3277.0779329679244),
 (24, 3256.8859131399468),
 (36, 3125.1667093517235),
 (17, 3102.1298483559267),
 (35, 3071.3574676198505),
 (33, 3051.426812916575),
 (13, 2973.6302536735307),
 (21, 2961.9215511844923),
 (27, 2845.071070710708),
 (50, 2797.5423042781067),
 (31, 2589.813488380271),
 (37, 2560.989916796452),
 (18, 2558.1572261417587),
 (30, 2545.3976594708033),
 (23, 2477.4845122572187),
 (12, 2337.554853961996),
 (49, 2312.091310097474),
 (20, 2254.878686883485),
 (40, 2194.97087897858),
 (6, 2101.6704167751864),
 (38, 2034.8886849113833),
 (8, 2016.5639313240113),
 (28, 1893.5451313640283),
 (45, 1778.9472723611868),
 (22, 1659.7042138658965)

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [46]:
user_items = sales_data.countByKey()
sorted(user_items.items(), key=lambda x: x[1], reverse=True)

[(36, 12),
 (21, 10),
 (4, 10),
 (10, 9),
 (39, 9),
 (19, 9),
 (50, 8),
 (18, 8),
 (8, 8),
 (3, 8),
 (15, 7),
 (48, 7),
 (22, 7),
 (2, 7),
 (26, 7),
 (31, 7),
 (37, 7),
 (35, 7),
 (16, 7),
 (9, 6),
 (20, 6),
 (40, 6),
 (33, 6),
 (49, 6),
 (44, 5),
 (28, 5),
 (25, 5),
 (24, 5),
 (47, 5),
 (38, 5),
 (11, 5),
 (5, 5),
 (29, 4),
 (46, 4),
 (12, 4),
 (32, 4),
 (42, 4),
 (14, 4),
 (1, 4),
 (30, 4),
 (45, 3),
 (43, 3),
 (34, 3),
 (13, 3),
 (7, 2),
 (17, 2),
 (41, 2),
 (6, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD, and extracting lineage and of an RDD in a spark application. We also used transformations and actions to perform calculations across RDDs on a distributed setup. Up next, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
